In [6]:
import os
import pandas as pd
import torch
import pydicom
import numpy as np
import albumentations as A
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from timm import create_model
import torch.nn as nn
import torch.optim as optim
from PIL import Image

# Path setup
BASE_DIR = "./dataset"
IMAGE_DIR = os.path.join(BASE_DIR, "images/images")
ANNOTATION_DIR = os.path.join(BASE_DIR, "annotations/annotations/tcia-lidc-xml")
CSV_FILE = os.path.join(BASE_DIR, "lidc_metadata.csv")

# Load metadata
metadata = pd.read_csv(CSV_FILE)
metadata['findings'] = metadata['findings'].fillna('')  # Replace NaN with empty string

# Label mapping (Nodules = 1, No Nodules = 0)
metadata['label'] = metadata['findings'].apply(lambda x: 1 if 'Nodules' in str(x) else 0)

# Convert to dictionary {image_id: label}
label_dict = dict(zip(metadata['image_id'], metadata['label']))

# Get all DICOM image file paths
image_paths = [os.path.join(IMAGE_DIR, fname) for fname in os.listdir(IMAGE_DIR) if fname.endswith(".dcm")]


# Custom Dataset Class (Supports DICOM Images)
class LIDCDataset(Dataset):
    def __init__(self, image_paths, label_dict, transform=None):
        self.image_paths = image_paths
        self.label_dict = label_dict
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        img_id = os.path.basename(img_path).replace(".dcm", "")

        # Load DICOM Image
        dicom_image = pydicom.dcmread(img_path)
        image = dicom_image.pixel_array  # Extract pixel data

        # Convert grayscale to RGB (if needed)
        if len(image.shape) == 2:  # Grayscale
            image = np.stack([image] * 3, axis=-1)  # Convert to 3-channel RGB

        # Normalize to 0-255
        image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255.0
        image = image.astype(np.uint8)

        # Get label (default: 0 if not found)
        label = self.label_dict.get(img_id, 0)

        # Apply transformations
        if self.transform:
            image = self.transform(image=image)["image"]

        # Convert to PyTorch tensor
        image = torch.tensor(image).permute(2, 0, 1)  # (H, W, C) → (C, H, W)
        label = torch.tensor(label, dtype=torch.float32)

        return image, label


# Define transformations
transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),  # Normalizing for RGB
    A.HorizontalFlip(p=0.5),
])

# Create dataset and dataloader
dataset = LIDCDataset(image_paths, label_dict, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


# Model Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model("resnet18", pretrained=True, num_classes=1).to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")

# Save model
torch.save(model.state_dict(), "lidc_model.pth")


C:\Users\Badari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Badari\.cache\huggingface\hub\models--timm--resnet18.a1_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Epoch 1, Loss: 0.5594189660302524
Epoch 2, Loss: 0.40461328009079245
Epoch 3, Loss: 0.306088341721173
Epoch 4, Loss: 0.24194999809922843
Epoch 5, Loss: 0.20392924932570294


In [7]:
import torch
import os
import pydicom
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from timm import create_model
import albumentations as A

# Path setup
BASE_DIR = "./dataset"
IMAGE_DIR = os.path.join(BASE_DIR, "images/images")
CSV_FILE = os.path.join(BASE_DIR, "lidc_metadata.csv")

# Load metadata
metadata = pd.read_csv(CSV_FILE)
metadata['findings'] = metadata['findings'].fillna('')
metadata['label'] = metadata['findings'].apply(lambda x: 1 if 'Nodules' in str(x) else 0)
label_dict = dict(zip(metadata['image_id'], metadata['label']))

# Get all DICOM image file paths
image_paths = [os.path.join(IMAGE_DIR, fname) for fname in os.listdir(IMAGE_DIR) if fname.endswith(".dcm")]

# Dataset Class (Same as Before)
class LIDCDataset(Dataset):
    def __init__(self, image_paths, label_dict, transform=None):
        self.image_paths = image_paths
        self.label_dict = label_dict
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        img_id = os.path.basename(img_path).replace(".dcm", "")

        # Load DICOM Image
        dicom_image = pydicom.dcmread(img_path)
        image = dicom_image.pixel_array  # Extract pixel data

        # Convert grayscale to RGB
        if len(image.shape) == 2:
            image = np.stack([image] * 3, axis=-1)

        # Normalize to 0-255
        image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255.0
        image = image.astype(np.uint8)

        # Get label
        label = self.label_dict.get(img_id, 0)

        # Apply transformations
        if self.transform:
            image = self.transform(image=image)["image"]

        # Convert to PyTorch tensor
        image = torch.tensor(image).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.float32)

        return image, label


# Define Transformations
transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

# Create Dataset and DataLoader for Evaluation
test_dataset = LIDCDataset(image_paths, label_dict, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model("resnet18", pretrained=True, num_classes=1).to(device)
model.load_state_dict(torch.load("lidc_model.pth"))
model.eval()

# Accuracy Calculation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = torch.sigmoid(model(images).squeeze())
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total * 100
print(f"Test Accuracy: {accuracy:.2f}%")


C:\Users\Badari\AppData\Local\Temp\ipykernel_22844\757425753.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lidc_model.pth"))


Test Accuracy: 95.03%
